In [2]:
#Import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
#Import total head dataframes
ChickensBroilers = pd.read_csv('Data/Selected/Imputed/ChickensBroilersTimeImputed.csv', dtype = 'str')
CowsBeef = pd.read_csv('Data/Selected/Imputed/CowsBeefTimeImputed.csv', dtype = 'str')
CowsMilk = pd.read_csv('Data/Selected/Imputed/CowsMilkTimeImputed.csv', dtype = 'str')
Hogs = pd.read_csv('Data/Selected/Imputed/HogsTimeImputed.csv', dtype = 'str')
ChickensLayers = pd.read_csv('Data/Selected/Imputed/ChickensLayersTimeImputed.csv', dtype = 'str')

for df in [ChickensBroilers, CowsBeef, CowsMilk, Hogs, ChickensLayers]:
    df['VALUE'] = df['VALUE'].astype('float')

In [52]:
#Create combined dataframes of all livestock
combined = pd.merge(left = pd.merge(left = pd.merge(left = pd.merge(left = ChickensBroilers[['YEAR', 'VALUE', 'county_state']].dropna(),
        right = CowsBeef[['YEAR', 'VALUE', 'county_state']].dropna(),
        on = ['YEAR', 'county_state'],
        suffixes = ['_ChickensBroilers', '_CowsBeef'],
        how = 'outer').fillna(0),
         right = CowsMilk[['YEAR', 'VALUE', 'county_state']].dropna(),
         on = ['YEAR', 'county_state'],
         how = 'outer').fillna(0),
         right = Hogs[['YEAR', 'VALUE', 'county_state']].dropna(),
         on = ['YEAR', 'county_state'],
         suffixes = ['_CowsMilk', '_Hogs'],
         how = 'outer').fillna(0),
         right = ChickensLayers[['YEAR', 'VALUE', 'county_state']].dropna(),
         on = ['YEAR', 'county_state'],
         how = 'outer').fillna(0).rename(columns = {'VALUE': 'VALUE_ChickensLayers'})

combined['Total Head'] = combined['VALUE_ChickensBroilers'] + combined['VALUE_CowsBeef'] + combined['VALUE_CowsMilk'] + combined['VALUE_Hogs'] + combined['VALUE_ChickensLayers']

combined['YEAR'] = combined['YEAR'].astype('int')

In [37]:
#Define function to plot max head in counties with max head across time
def max_plot(df, title):
    max_counties = df.loc[df[df['AGG_LEVEL_DESC'] == 'COUNTY'].groupby("YEAR")["VALUE"].idxmax()][['YEAR', 'VALUE', 'county_state']].sort_values(by = 'YEAR')

    plt.figure(figsize = (7, 5))
    plt.title(f"Maximum {title} by County")
    plt.ylabel("Head")
    plt.plot(max_counties['YEAR'], max_counties['VALUE'], color = 'k')
    for county in max_counties['county_state'].unique():
        plt.bar(max_counties[max_counties['county_state'] == county]['YEAR'],
                max_counties[max_counties['county_state'] == county]['VALUE'], 
                label = county,
                width = 1);
    plt.xticks(['1980', '1990', '2000', '2010', '2020'])
    plt.xlabel("Year")
    plt.legend()
    plt.tight_layout();
    plt.savefig(f'Plots/max_plot_{title}.jpg')
    plt.close();

In [38]:
#Plot Maximum head per county for each livestock type
max_plot(ChickensBroilers, "Broiler Chickens")
max_plot(CowsBeef, "Beef Cows")
max_plot(CowsMilk, "Milk Cows")
max_plot(Hogs, "Hogs")
max_plot(ChickensLayers, "Layer Chickens")